In [1]:
import eikon as ek
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
from fredapi import Fred


fred = Fred(api_key = "4a017ca39a1f96774f9587e5956bfd6b") # we can all use the same key
#ek.set_app_key("320e7257c23945729695a106cedb2b9cb72d3207") # Elena
#ek.set_app_key("320e7257c23945729695a106cedb2b9cb72d3207") # Elena
ek.set_app_key("69de0ce7c5e2468e90a6058ce79f6558ebe3ee30") # Alexander
#ek.set_app_key("8879958aee0d4e68894df7ee96fe7c192adc17fc") # Mohandas
#ek.set_app_key("f47c330480d74c598b7e8ebc2539424e91764dd8") # Ginevra

https://community.developers.refinitiv.com/questions/73493/get-eps-historical-data-for-stocks.html

# Economic Story

**TR.RevenueActValue.date** - prints the datetime

## Part 1: Accuracy

1. **TR.EPSActValue** - The company's actual value normalized to reflect the I/B/E/S default currency and corporate actions (e.g. stock splits). Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.

2. **TR.EPSMean** - The statistical average of all broker estimates determined to be on the majority accounting basis. Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.

--> this is a analyst forecast variable

3. **TR.EPSActSurprise** - The difference between the actual and the last mean of the period, expressed as a percentage. Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.

--> forecast error between actual EPS and TR.EPSMean

<img src="./Pictures/Screenshot 2023-02-15 185222.png" width=500 height=300 />

## Part 2: Factors

### Firm charateristics

1. **TR.RevenueActValue** - The company's actual value normalized to reflect the I/B/E/S default currency and corporate actions (e.g. stock splits). Revenue (or Sales) is a corporation's net revenue, generally derived from core business activities. For non-financial companies, the calculation of net revenue (or net turnover) in most markets generally involves subtracting transportation and related operational costs from gross revenue/sales. Revenue recognition practices vary significantly from market to market, though generally the recording of revenue is based upon sales invoices issued (or anticipated for forecast purposes) during the accounting period.
2. **TR.F.EV** - Enterprise Value is the Company's total economic value OR the theoretical takeover price, often used as an alternative to market capitalization. Enterprise Value represents Market Capitalization TR.F.MktCap + Debt including Preferred Equity & Minority Interest - Total [TR.F.DebtlnclPrefEqMinlntrTot] (-) Cash & Short Term Investments - Total [TR.F.CashSTlnvstTot]. Market Capitalization [TR.F.MktCap] is required. The data item is calculated for all periodicities. It is applicable to all industries.
3. **TR.TotalReturn3Mo** - The 3 months total return incorporates the price Change and any relevant dividends for the last 3 months.
4. **TR.EVTosales** - EV represents the sum of Market Capitalization, Total Debt, Preferred Stock and Minority Interest minus Cash and Short Term Investments for the most recent fiscal period. Market Cap is calculated by multiplying Current Total Shares Outstanding by Latest Close Price. Sales is LTM Total Revenue. EV to Sales ratio is not calculated when LTM Sales is less than or equal to Zero.
5. **TR.F.MktCap** Market Cap
6. **TR.F.NetIncAfterTax** Net Income after Tax [SIAT] represents the income/expense after all operating and non-operating income and expense, reserves, income taxes, but before equity in earnings, minority interest, extraordinary items, after-tax adjustments, discontinued operations and preferred dividends. Applicable to all Industries. Net Income after Tax [SIAT] includes: • Net Income after Tax [XIAT]
    **loss firm status** - -1 for profit firm, +1 for loss firm based on net income after tax
7. **TR.Volume** - Volume for the latest trading day. For stock exchanges that trade share by share, Volume is number of shares that traded on the trade date. For stock exchanges that trade in lots, Volume is divided by Lotsize, if the Lotsize is greater than one.

### Analysts
1. **TR.NumberOfAnalysts** - Number of sell-side analysts covering the security.
2. **TR.RecMean** - Recommendation Numeric Mean based on the Standard Scale Of Strong Buy (1 ), Buy (2), Hold (3), Sell (4), and Strong Sell (5).
3. change in recommendation
4. **TR.EPSCoefficientOfVariation**
5. **TR.priceTargetStdDev** The statistical standard deviation of all estimates included in the summary calculation- Price Target is the projected price level forecasted by the analyst Within a specific time nonzon-

### Macro characteristics

1. **TR.WACCInflationAdjRiskFreeRate** -  Inflation adjusted risk free rate used is Of the primary index for the domicile country Of the company. Inflation adjusted risk free rate (Rf) is calculated from the US 10-year treasury yield plus the difference between in the 10-year forecasted inflation rate between the given country and the US.



### Return Potential

1. **TR.PriceTargetMean** The statistical average of all broker estimates determined to be on the majority accounting basis. Price target is the projected price level forecasted by the analyst within a specific time horizon.

1. **TR.PriceClose** The latest available closing price. If there are no trades for the most recent completed tradable day, the most recent prior tradable day with trading activity is used, provided the last tradable day for the instrument is within 378 completed calendar days.

### More difficult Factors (yearly):

#### ESG

1. **TR.TRESGScore** Refinitiv ESG Score is an overall company score based on the self-reported information in the environmental, social and corporate governance pillars.

#### Proportion of independent Directors

1. **TR.AnalyticIndepBoard** - Independent Board Members - Percentage of independent board members as reported by the company.

2. **TR.GovernancePillarScore** - Governance Pillar Score - The corporate governance pillar measures a company's systems and processes, which ensure that its board members and executives act in the best interests of its long term shareholders. It reflects a company's capacity, through its use of best management practices, to direct and control its rights and responsibilities through the creation of incentives, as well as checks and balances in order to generate long term shareholder value.

## Part 3: Regression

### continue..

In [2]:
economic_story = ['TR.EPSactValue.date', 'TR.EPSActValue', "TR.EPSMean", "TR.EPSActSurprise", "TR.EPSCoefficientOfVariation", "TR.F.MktCap", 'TR.RevenueActValue', "TR.F.EV",  "TR.NumberOfAnalysts", "TR.RecMean", "TR.RecMean(SDate=-2,EDate=-41)", "TR.WACCInflationAdjRiskFreeRate", "TR.F.NetIncAfterTax", "TR.PriceTargetMean", "TR.priceTargetStdDev", "TR.PriceClose", "TR.EVTosales", "TR.TotalReturn3Mo", "TR.Volume"]
#TR.GICSIndustryGroup

# Whole Dataframe

Remove outliers in this step for EPS actual surprise


In [3]:
df, e = ek.get_data('0#.SPX', economic_story, parameters = {'SDate':'-1','EDate':'-40','Period':'FQ0','Frq':'FQ'})
df["Date"] = pd.to_datetime(df["Date"]).dt.tz_localize(None)

C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\pandas\core\dtypes\cast.py:1134: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\pandas\core\dtypes\cast.py:1153: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


In [4]:
df

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Enterprise Value,Number of Analysts,Recommendation - Mean (1-5),Recommendation - Mean (1-5),"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,Price Target - Mean,Price Target - Standard Deviation,Price Close,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume
0,POOL.OQ,2022-10-20 07:00:00,4.78,4.5875,4.196,3.84370572207084,12598134372.299999,1615339000.0,14061600372.299999,10,2.27273,2.2,3.747421,189965000.0,360.11111,24.95082,318.21,2.383926,-9.16437517579112,238539
1,POOL.OQ,2022-07-21 07:00:00,7.63,7.517,1.503,3.62524943461487,14057953705.290001,2055818000.0,15561870705.290001,9,2.2,2.0,3.092855,307205000.0,435.375,57.57373,351.23,2.749481,-21.6615695848725,185584
2,POOL.OQ,2022-04-21 07:00:00,4.23,3.14867,34.342,6.63073615208961,16967115034.200001,1412650000.0,18436823034.200001,9,2.0,2.3,2.325202,179203000.0,519.88889,37.49206,422.85,3.42288,-25.1606642775324,290897
3,POOL.OQ,2022-02-17 07:00:00,2.63,1.875,40.267,3.26986666666667,22689791586.0,1035557000.0,23848820586.0,8,2.3,2.3,1.515266,107542000.0,571.0,41.33833,566.0,4.504382,30.4897054542077,82349
4,POOL.OQ,2021-10-21 07:00:00,4.51,3.84833,17.194,2.83629522416217,17420199822.66,1411448000.0,17699543822.66,8,2.3,2.3,1.527139,184573000.0,545.28571,50.48823,434.41,3.6844,-5.13032169564558,157740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,AVY.N,2014-01-31 08:30:00,0.69,0.68,1.471,3.20441176470588,4877167502.24,1583900000.0,5553067502.24,8,2.33333,2.625,<NA>,42100000.0,54.625,3.42555,50.19,0.920659,16.0038520663565,157761
20116,AVY.N,2013-10-25 08:30:00,0.69,0.63833,8.095,2.77599360832171,4273414353.46,1504900000.0,5029514353.46,7,2.625,2.875,<NA>,62000000.0,47.71429,3.4523,43.52,0.903176,2.46762008841375,236291
20117,AVY.N,2013-07-23 08:30:00,0.71,0.7025,1.068,1.55160142348754,4258229328.84,1552300000.0,5436229328.84,8,2.875,2.875,<NA>,70800000.0,45.0,4.30946,42.76,0.906078,-0.054218308940701,0
20118,AVY.N,2013-04-24 08:30:00,0.59,0.57571,2.482,2.04616907818172,4309428200.46,1498900000.0,5459128200.46,7,2.875,3.0,<NA>,66800000.0,42.57143,3.92272,43.07,0.895787,24.1487263146973,0


In [5]:
unemployment_rate = fred.get_series('UNRATE', observation_start='2012-12-01', observation_end='2022-11-01')

In [6]:
unemployment_rate = pd.DataFrame(unemployment_rate)
unemployment_rate['index'] = unemployment_rate.index
unemployment_rate = unemployment_rate.reset_index(drop=True)
unemployment_rate.columns = ["Unemployment rate", "Date"]
unemployment_rate["Date"] = pd.to_datetime(unemployment_rate["Date"])
unemployment_rate

,Unemployment rate,Date
0,7.9,2012-12-01
1,8.0,2013-01-01
2,7.7,2013-02-01
3,7.5,2013-03-01
4,7.6,2013-04-01
...,...,...
115,3.5,2022-07-01
116,3.7,2022-08-01
117,3.5,2022-09-01
118,3.7,2022-10-01


In [7]:
#DO NOT DELETE!
df_for_accuracy_analysis = df.copy()
df_for_accuracy_analysis.to_csv("Dataframes/accuracy_analysis_dataframe.csv", index=False)

In [8]:
df['New Date'] = pd.Series()

for i in range(len(df)):
    if df.loc[i, "Date"].month == 1:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=1, day=1)
    elif df.loc[i, "Date"].month == 2:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=1, day=1)
    elif df.loc[i, "Date"].month == 3:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=1, day=1)
    elif df.loc[i, "Date"].month == 4:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=4, day=1)
    elif df.loc[i, "Date"].month == 5:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=4, day=1)
    elif df.loc[i, "Date"].month == 6:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=4, day=1)
    elif df.loc[i, "Date"].month == 7:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=7, day=1)
    elif df.loc[i, "Date"].month == 8:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=7, day=1)
    elif df.loc[i, "Date"].month == 9:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=7, day=1)
    elif df.loc[i, "Date"].month == 10:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=10, day=1)
    elif df.loc[i, "Date"].month == 11:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=10, day=1)
    elif df.loc[i, "Date"].month == 12:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=10, day=1)

df.insert(2, "New Date", df.pop("New Date"))
df["New Date"] = pd.to_datetime(df["New Date"]).dt.date
df

C:\Users\alexa\AppData\Local\Temp\ipykernel_16300\2409114918.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['New Date'] = pd.Series()


,Instrument,Date,New Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Enterprise Value,...,Recommendation - Mean (1-5),Recommendation - Mean (1-5),"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,Price Target - Mean,Price Target - Standard Deviation,Price Close,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume
0,POOL.OQ,2022-10-20 07:00:00,2022-10-01,4.78,4.5875,4.196,3.84370572207084,12598134372.299999,1615339000.0,14061600372.299999,...,2.27273,2.2,3.747421,189965000.0,360.11111,24.95082,318.21,2.383926,-9.16437517579112,238539
1,POOL.OQ,2022-07-21 07:00:00,2022-07-01,7.63,7.517,1.503,3.62524943461487,14057953705.290001,2055818000.0,15561870705.290001,...,2.2,2.0,3.092855,307205000.0,435.375,57.57373,351.23,2.749481,-21.6615695848725,185584
2,POOL.OQ,2022-04-21 07:00:00,2022-04-01,4.23,3.14867,34.342,6.63073615208961,16967115034.200001,1412650000.0,18436823034.200001,...,2.0,2.3,2.325202,179203000.0,519.88889,37.49206,422.85,3.42288,-25.1606642775324,290897
3,POOL.OQ,2022-02-17 07:00:00,2022-01-01,2.63,1.875,40.267,3.26986666666667,22689791586.0,1035557000.0,23848820586.0,...,2.3,2.3,1.515266,107542000.0,571.0,41.33833,566.0,4.504382,30.4897054542077,82349
4,POOL.OQ,2021-10-21 07:00:00,2021-10-01,4.51,3.84833,17.194,2.83629522416217,17420199822.66,1411448000.0,17699543822.66,...,2.3,2.3,1.527139,184573000.0,545.28571,50.48823,434.41,3.6844,-5.13032169564558,157740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,AVY.N,2014-01-31 08:30:00,2014-01-01,0.69,0.68,1.471,3.20441176470588,4877167502.24,1583900000.0,5553067502.24,...,2.33333,2.625,<NA>,42100000.0,54.625,3.42555,50.19,0.920659,16.0038520663565,157761
20116,AVY.N,2013-10-25 08:30:00,2013-10-01,0.69,0.63833,8.095,2.77599360832171,4273414353.46,1504900000.0,5029514353.46,...,2.625,2.875,<NA>,62000000.0,47.71429,3.4523,43.52,0.903176,2.46762008841375,236291
20117,AVY.N,2013-07-23 08:30:00,2013-07-01,0.71,0.7025,1.068,1.55160142348754,4258229328.84,1552300000.0,5436229328.84,...,2.875,2.875,<NA>,70800000.0,45.0,4.30946,42.76,0.906078,-0.054218308940701,0
20118,AVY.N,2013-04-24 08:30:00,2013-04-01,0.59,0.57571,2.482,2.04616907818172,4309428200.46,1498900000.0,5459128200.46,...,2.875,3.0,<NA>,66800000.0,42.57143,3.92272,43.07,0.895787,24.1487263146973,0


In [9]:
df = df.drop("Date", axis = 1)
df.rename(columns={"New Date": "Date"}, inplace=True)
df["Date"] = pd.to_datetime(df["Date"])
df

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Enterprise Value,Number of Analysts,Recommendation - Mean (1-5),Recommendation - Mean (1-5),"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,Price Target - Mean,Price Target - Standard Deviation,Price Close,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume
0,POOL.OQ,2022-10-01,4.78,4.5875,4.196,3.84370572207084,12598134372.299999,1615339000.0,14061600372.299999,10,2.27273,2.2,3.747421,189965000.0,360.11111,24.95082,318.21,2.383926,-9.16437517579112,238539
1,POOL.OQ,2022-07-01,7.63,7.517,1.503,3.62524943461487,14057953705.290001,2055818000.0,15561870705.290001,9,2.2,2.0,3.092855,307205000.0,435.375,57.57373,351.23,2.749481,-21.6615695848725,185584
2,POOL.OQ,2022-04-01,4.23,3.14867,34.342,6.63073615208961,16967115034.200001,1412650000.0,18436823034.200001,9,2.0,2.3,2.325202,179203000.0,519.88889,37.49206,422.85,3.42288,-25.1606642775324,290897
3,POOL.OQ,2022-01-01,2.63,1.875,40.267,3.26986666666667,22689791586.0,1035557000.0,23848820586.0,8,2.3,2.3,1.515266,107542000.0,571.0,41.33833,566.0,4.504382,30.4897054542077,82349
4,POOL.OQ,2021-10-01,4.51,3.84833,17.194,2.83629522416217,17420199822.66,1411448000.0,17699543822.66,8,2.3,2.3,1.527139,184573000.0,545.28571,50.48823,434.41,3.6844,-5.13032169564558,157740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,AVY.N,2014-01-01,0.69,0.68,1.471,3.20441176470588,4877167502.24,1583900000.0,5553067502.24,8,2.33333,2.625,<NA>,42100000.0,54.625,3.42555,50.19,0.920659,16.0038520663565,157761
20116,AVY.N,2013-10-01,0.69,0.63833,8.095,2.77599360832171,4273414353.46,1504900000.0,5029514353.46,7,2.625,2.875,<NA>,62000000.0,47.71429,3.4523,43.52,0.903176,2.46762008841375,236291
20117,AVY.N,2013-07-01,0.71,0.7025,1.068,1.55160142348754,4258229328.84,1552300000.0,5436229328.84,8,2.875,2.875,<NA>,70800000.0,45.0,4.30946,42.76,0.906078,-0.054218308940701,0
20118,AVY.N,2013-04-01,0.59,0.57571,2.482,2.04616907818172,4309428200.46,1498900000.0,5459128200.46,7,2.875,3.0,<NA>,66800000.0,42.57143,3.92272,43.07,0.895787,24.1487263146973,0


In [10]:
df = pd.merge(df, unemployment_rate, on='Date', how='left')
df

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Enterprise Value,Number of Analysts,...,Recommendation - Mean (1-5),"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,Price Target - Mean,Price Target - Standard Deviation,Price Close,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,Unemployment rate
0,POOL.OQ,2022-10-01,4.78,4.5875,4.196,3.84370572207084,12598134372.299999,1615339000.0,14061600372.299999,10,...,2.2,3.747421,189965000.0,360.11111,24.95082,318.21,2.383926,-9.16437517579112,238539,3.7
1,POOL.OQ,2022-07-01,7.63,7.517,1.503,3.62524943461487,14057953705.290001,2055818000.0,15561870705.290001,9,...,2.0,3.092855,307205000.0,435.375,57.57373,351.23,2.749481,-21.6615695848725,185584,3.5
2,POOL.OQ,2022-04-01,4.23,3.14867,34.342,6.63073615208961,16967115034.200001,1412650000.0,18436823034.200001,9,...,2.3,2.325202,179203000.0,519.88889,37.49206,422.85,3.42288,-25.1606642775324,290897,3.6
3,POOL.OQ,2022-01-01,2.63,1.875,40.267,3.26986666666667,22689791586.0,1035557000.0,23848820586.0,8,...,2.3,1.515266,107542000.0,571.0,41.33833,566.0,4.504382,30.4897054542077,82349,4.0
4,POOL.OQ,2021-10-01,4.51,3.84833,17.194,2.83629522416217,17420199822.66,1411448000.0,17699543822.66,8,...,2.3,1.527139,184573000.0,545.28571,50.48823,434.41,3.6844,-5.13032169564558,157740,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,AVY.N,2014-01-01,0.69,0.68,1.471,3.20441176470588,4877167502.24,1583900000.0,5553067502.24,8,...,2.625,<NA>,42100000.0,54.625,3.42555,50.19,0.920659,16.0038520663565,157761,6.6
20116,AVY.N,2013-10-01,0.69,0.63833,8.095,2.77599360832171,4273414353.46,1504900000.0,5029514353.46,7,...,2.875,<NA>,62000000.0,47.71429,3.4523,43.52,0.903176,2.46762008841375,236291,7.2
20117,AVY.N,2013-07-01,0.71,0.7025,1.068,1.55160142348754,4258229328.84,1552300000.0,5436229328.84,8,...,2.875,<NA>,70800000.0,45.0,4.30946,42.76,0.906078,-0.054218308940701,0,7.3
20118,AVY.N,2013-04-01,0.59,0.57571,2.482,2.04616907818172,4309428200.46,1498900000.0,5459128200.46,7,...,3.0,<NA>,66800000.0,42.57143,3.92272,43.07,0.895787,24.1487263146973,0,7.6


In [11]:
df.to_csv("Dataframes/economic_story_dataframe.csv", index=False)

In [12]:
#yearly = ["TR.TRESGScore.date", "TR.TRESGScore", "TR.GovernancePillarScore", "TR.AnalyticIndepBoard"]

In [13]:
#economic_story_yearly, e = ek.get_data('0#.SPX', yearly, parameters = {'SDate':'0','EDate':'-20','Period':'FY0','Frq':'FY'})

In [14]:
#economic_story_yearly["Date"] = pd.to_datetime(economic_story_yearly["Date"]).dt.date
#economic_story_yearly = economic_story_yearly.dropna()
#economic_story_yearly

In [15]:
#economic_story_yearly.to_csv("yearly_dataframe.csv", index=False)